<a href="https://colab.research.google.com/github/arnav2/Crowd_Counting/blob/master/Crowd_Counting_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv
import numpy as np
import random
import torch
import torch.nn as nn
import torch.utils.data
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import torch.nn.functional as F
import time
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, models, transforms
import os

In [0]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [0]:
# Total data I have is 1600 images. I ensure they are set to the same size of 224
def data_collection(batch_size):
  data_dir = '/content/drive/My Drive/APS360-Crowd_Counting/'

  data_transform = transforms.Compose([transforms.RandomResizedCrop(224), 
                                        transforms.ToTensor()])

  total_data = datasets.ImageFolder(data_dir, transform=data_transform)
  valid_data = datasets.ImageFolder(data_dir, transform=data_transform)
  test_data  = datasets.ImageFolder(data_dir, transform=data_transform)
  
  
  data_size = len(total_data)
  indices = list(range(data_size))
  
  valTrainRatio = 0.2
  split1 = int(np.floor(valTrainRatio*data_size)) # First split
  split2 = split1*2

  #Shuffle the indices randomly
  np.random.seed(360)
  np.random.shuffle(indices)

  train_idx, valid_idx,test_idx = indices[split2:], indices[split1:split2], indices[:split1]
  train_sampler = SubsetRandomSampler(train_idx)
  valid_sampler = SubsetRandomSampler(valid_idx)
  test_sampler = SubsetRandomSampler(test_idx)
  # For some reason cannot shuffle the entire dataset here 
  train_loader = torch.utils.data.DataLoader(total_data,batch_size=batch_size,sampler= train_sampler)
  val_loader = torch.utils.data.DataLoader(valid_data,  batch_size=batch_size,sampler= valid_sampler)
  test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,sampler= test_sampler )
  print(len(train_loader))
  return train_loader,val_loader,test_loader

In [0]:
class MyCoConvu(nn.Sequential):
  def __init__(self, input_feat, output_feat, kernel_size, padding):
    super(MyCoConvu, self).__init__(
        nn.Conv2d(input_feat, output_feat, kernel_size=kernel_size, padding=padding),
        nn.BatchNorm2d(output_feat, eps=1e-03),
        nn.ReLU()
    )

In [0]:
# The deafault stride is 2 for kernel size 2 for max pooling
class MCNNNet(nn.Module):

  def __init__(self):
    super(MCNNNet, self).__init__()
    self.name = "baselineModel"
    features_1 = [ 
                MyCoConvu(3, 16,kernel_size=1 , padding=0) ,
                nn.MaxPool2d(kernel_size=2),
                MyCoConvu(16, 32,kernel_size=7 , padding = 3),
                nn.MaxPool2d(kernel_size=2),
                MyCoConvu(32, 16,kernel_size=7 , padding = 3),
                MyCoConvu(16, 8,kernel_size=7, padding = 3)]
                
    features_2 = [
                MyCoConvu(3, 20,kernel_size=7,padding = 3),
                nn.MaxPool2d(kernel_size=2),
                MyCoConvu(20, 40,kernel_size=5, padding = 2),
                nn.MaxPool2d(kernel_size=2),
                MyCoConvu(40, 20,kernel_size=5 , padding = 2),
                MyCoConvu(20, 10,kernel_size=5, padding = 2)]
    features_3 = [
                MyCoConvu(3, 24,kernel_size=5, padding = 2),
                nn.MaxPool2d(kernel_size=2),
                MyCoConvu(24, 48,kernel_size=3, padding = 1),
                nn.MaxPool2d(kernel_size=2),
                MyCoConvu(48, 24,kernel_size=3, padding = 1),
                MyCoConvu(24, 12,kernel_size=3, padding = 1)]
    final_conv = [MyCoConvu(30, 1,kernel_size=1, padding = 0)]
    self.features_1 = nn.Sequential(*features_1)
    self.features_2 = nn.Sequential(*features_2)
    self.features_3 = nn.Sequential(*features_3)
    self.final_conv = nn.Sequential(*final_conv)
  def forward(self, x):
    x1 = self.features_1(x)
    print(x1.shape)
    x2 = self.features_2(x)
    print(x2.shape)
    x3 = self.features_3(x)
    print(x3.shape)
    x = torch.cat((x1,x2,x3),1)
    x = self.final_conv(x)
    return x

In [0]:
def train(model, train_loader, val_loader, test_loader, batch_size=64, num_iters=100, learn_rate=0.01, weight_decay=0):
    
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=learn_rate, momentum=0.9, weight_decay=weight_decay)

    iters, losses, train_acc, val_acc = [], [], [], []
    use_cuda = 1
    
    if use_cuda and torch.cuda.is_available():
      model = model.cuda()
    # training
    
    # training
    n = 0 # the number of iterations
    while True:
        if n >= num_iters:
            break
        n += 1
        for imgs, labels in iter(train_loader):
          #############################################
          print(labels)
          #To Enable GPU Usage
          if use_cuda and torch.cuda.is_available():
            imgs = imgs.cuda()
            labels = labels.cuda()
          print("Shape of labels is : ", labels.shape)
          #############################################
          model.train() #*****************************#
          out = model(imgs)             # forward pass
          print("Shape of output is : " , out.shape)
          loss = criterion(out, labels) # compute the total loss
          loss.backward()               # backward pass (compute parameter updates)
          optimizer.step()              # make the updates for each parameter
          optimizer.zero_grad()         # a clean up step for PyTorch

        # save the current training information
        if n % 10 == 9:
          iters.append(n)
          losses.append(float(loss)/batch_size)        # compute *average* loss
          train_acc.append(get_accuracy(model, train_loader)) # compute training accuracy 
          val_acc.append(get_accuracy(model, val_loader))   # compute validation accuracy
          
          print(("Epoch {}: Train loss: {}, Train Accuracy: {} |"+
                 "Validation accuracy: {}").format(
                   n + 1,
                   losses[-1],
                   train_acc[-1],
                   val_acc[-1]))
          #Add checkpoints here to store the model to a file with frozen weights
          
          model_path = get_model_name(model.name, batch_size, learn_rate,weight_decay,n)
          torch.save(model.state_dict(), model_path)
          
    # plotting
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.title("Training Curve")
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")

    plt.subplot(1,2,2)
    plt.title("Training Curve")
    plt.plot(iters, train_acc, label="Train")
    plt.plot(iters, val_acc, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()
    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))

model = MCNNNet()
train_loader, val_loader, test_loader = data_collection(64)
train(model, train_loader, val_loader, test_loader, batch_size=64, num_iters=1, learn_rate=0.01, weight_decay=0)

15
tensor([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0])
Shape of labels is :  torch.Size([64])
torch.Size([64, 8, 226, 226])
torch.Size([64, 10, 226, 226])


KeyboardInterrupt: ignored